In [1]:
import numpy as np
import math

# 共同住宅の躯体性能の簡易評価法

## I. Orientation Class

### Normal Parts

In [2]:
class OrientationValue():
    def __init__( self, v, o ):
        # 'v' is Value and 'o' is Orientation 
        if not ( o == 'D0' or o == 'D90' or o == 'D180' or o == 'D270' or o == 'top' or o == 'bottom' ):
            raise ValueError
        self.v = v
        self.o = o
    def __str__( self ):
        return {
            'D0'     : 'D0     : ',
            'D90'    : 'D90    : ',
            'D180'   : 'D180   : ',
            'D270'   : 'D270   : ',
            'top'    : 'top    : ',
            'bottom' : 'bottom : ',
               }[self.o] + str(self.v)

In [3]:
class HorizontalOrientation():
    def __init__( self, d0, d90, d180, d270 ):
        self.d0   = d0
        self.d90  = d90
        self.d180 = d180
        self.d270 = d270
        
    def getMembers( self ):
        return [ self.d0, self.d90, self.d180, self.d270 ]

#### Example

In [4]:
try:
    OrientationValue( 0.12, 'D45')
except ValueError:
    print('error')

error


In [5]:
print( OrientationValue( 0.12, 'D90' ) )

D90    : 0.12


### Heat Bridges

In [6]:
class OrientationValueHB():
    def __init__( self, v, o1, o2 ):
        # 'v' is Value, 'o1' is Orientation, 'o2' is Orientation 
        def ValueCheck( o ):
            if not ( o == 'D0' or o == 'D90' or o == 'D180' or o == 'D270' or o == 'top' or o == 'bottom' ):
                raise ValueError
        ValueCheck( o1 )
        ValueCheck( o2 )
        self.v = v
        self.o1 = o1
        self.o2 = o2
    def __str__( self ):
        return {
            'D0'     : 'D0     + ',
            'D90'    : 'D90    + ',
            'D180'   : 'D180   + ',
            'D270'   : 'D270   + ',
            'top'    : 'top    + ',
            'bottom' : 'bottom + ',
               }[self.o1] + {
            'D0'     : 'D0     : ',
            'D90'    : 'D90    : ',
            'D180'   : 'D180   : ',
            'D270'   : 'D270   : ',
            'top'    : 'top    : ',
            'bottom' : 'bottom : ',
               }[self.o2] + str(self.v)

In [7]:
try:
    OrientationValueHB( 0.12, 'D45', 'D90' )
except ValueError:
    print('error')

error


In [8]:
try:
    OrientationValueHB( 0.12, 'D90', 'D45')
except ValueError:
    print('error')

error


In [9]:
print( OrientationValueHB( 0.12, 'top', 'D90' ) )

top    + D90    : 0.12


## II. Input Data

### 1) 住棟全体

| No. | 入力項目 | 単位 | 記号 | 備考 |
| --- | --- | --- | --- | --- |
| G1 | 地域の区分 | - | - | 1～8地域 |
| G2 | 建物の構造 | - | - | RC, S, W |
| G3 | 建物の階数 | 階 | $N$ | - |
| G4 | 建物全体の高さ | m | $H_{total}$ | - |
| G5 | 主開口部の方位 | - | - | 8方位 |

````
d_General = {
    'Region'               : 6,    # 地域の区分
    'Structure'            : 'RC', # 建物の構造
    'LevelsNumber'         : 3,    # 建物の階数
    'Height'               : 9.1,  # 建物の高さ(m)
    'OpeningMainDirection' : 'S',  # 主開口部の方位
}
````

### 2) 専有部

| No. | 入力項目 | 単位 | 記号 | 備考 |
| --- | --- | --- | --- | --- |
| P1 | 各階の住戸数 | 戸 | $n$ | 階ごとに入力する |
| P2 | 各階の床面積の合計 | m<sup>2</sup>| $A_{f,total} $ | 階ごとに入力する |
| P3 | 各階の外周長 外気と接する部分 | m | | 階ごとに入力する |
| P4 | 各階の外周長 空調された共用部や非住宅部と接する部分 | m | | 階ごとに入力する |
| P5 | 各階の外周長 住戸と接する部分 | m | | 階ごとに入力する (住棟を複数の方位に分けて計算する場合のみ入力) |
| P6 | 各階の窓面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| P7 | 各階(最上階を除く)の屋根面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| P8 | 各階(最下階を除く)の外気床の面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| P9 | 外壁の仕様 熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P10 | 外壁の仕様 断熱位置 | - | | inside=内断熱, outside=外断熱, non=無断熱 から選択 |
| P11 | 外壁の仕様 断熱補強の仕様 | - | | spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択 |
| P12 | 屋根の仕様 熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P13 | 屋根の仕様 断熱位置 | - | | inside=内断熱, outside=外断熱, non=無断熱 から選択 |
| P14 | 屋根の仕様 断熱補強の仕様 | - | | spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択 |
| P15 | 最下階床の仕様 熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P16 | 最下階床の仕様 断熱位置 | - | | inside=内断熱, outside=外断熱, non=無断熱 から選択 |
| P17 | 最下階床の仕様 断熱補強の仕様 | - | | spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択 |
| P18 | 界壁の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P19 | 界床の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P20 | 窓の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P21 | 窓の日射熱取得率(η値) | - | | |

````
d_PrivateArea = {
    'HouseNumber'         : [4, 4, 4],                # 各階の住戸数
    'FloorArea'           : [162.64, 162.64, 162.64], # 各階の床面積の合計(m2)
    'PeriLengthOutside'   : [66.56, 66.56, 66.56],    # 各階の外周長 外気と接する部分(m)
    'PeriLengthInside'    : [0.0, 0.0, 0.0],          # 各階の外周長 空調された共用部や非住宅部と接する部分(m)
    'PeriLengthPrivate'   : [0.0, 0.0, 0.0],          # 各階の外周長 住戸と接する部分(住棟を複数の方位に分けて計算する場合のみ)(m)
    'WindowArea'          : [30.48, 30.64, 30.64],    # 各階の窓面積の合計(m2)
    'RoofArea'            : [0.0, 0.0, None],         # 各階(最上階を除く)の屋根面積の合計(m2)
    'OutsideFloorArea'    : [None, 0.0, 0.0],         # 各階(最下階を除く)の外気床の面積の合計(m2)
    'Wall'                : {                         # 外壁の仕様
        'UValue' : 0.785,    # 熱貫流率(U値)(W/m2K)
        'InsPos' : 'inside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
        'HBSpec' : 'spec1'   # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
    },
    'Roof'                : {                      # 屋根の仕様
        'UValue' : 0.417,     # 熱貫流率(U値)(W/m2K)
        'InsPos' : 'outside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
        'HBSpec' : 'spec1'    # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
    },
    'Floor'               : {                      # 最下階床の仕様
        'UValue' : 0.507,     # 熱貫流率(U値)(W/m2K)
        'InsPos' : 'outside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
        'HBSpec' : 'null'     # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
    },
    'InsideWallUValue'    : 2.34,                  # 界壁の熱貫流率(U値)(W/m2K)
    'InsideFloorUValue'   : 1.796,                 # 界床の熱貫流率(U値)(W/m2K)
    'WindowUValue'        : 4.65,                  # 窓の熱貫流率(U値)(W/m2K)
    'WindowEtaValue'      : 0.79                   # 窓の日射熱取得率(η値)    
}
````

### 3) 共用部

| No. | 入力項目 | 単位 | 記号 | 備考 |
| --- | --- | --- | --- | --- |
| C1 | 各階の共用部・非住宅部の床面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| C2 | 各階(最上階を除く)の屋根面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| C3 | 各階(最下階を除く)の外気床の面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| C4 | 外壁の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| C5 | 界床の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| C6 | 外気床の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| C7 | 窓の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| C8 | 窓の日射熱取得率(η値) | - | |  |

````
d_PublicArea = {
    'FloorArea'          : [68.82, 59.7, 59.7], # 各階の共用部・非住宅部の床面積の合計(m2)
    'RoofArea'           : [9.125, 0.0, None],  # 各階(最上階を除く)の屋根面積の合計(m2)
    'OutsideFloorArea'   : [None, 0.0, 0.0],    # 各階(最下階を除く)の外気床の面積の合計(m2)
    'WallUValue'         : 0.00,                # 外壁の熱貫流率(U値)(W/m2K)
    'InsideFloorUValue'  : 0.00,                # 界床の熱貫流率(U値)(W/m2K)
    'OutsideFloorUValue' : 0.00,                # 外気床の熱貫流率(U値)(W/m2K)
    'WindowUValue'       : 0.00,                # 窓の熱貫流率(U値)(W/m2K)
    'WindowEtaValue'     : 0.00                 # 窓の日射熱取得率(η値)
}
````

````
d = {
    "General"     : d_General,
    "PrivateArea" : d_PrivateArea,
    "PublicArea"  : d_PublicArea
}
````

## IV. Functions

### 1. 専有部における各階の面積・長さ

### 1.1 屋根面積

階$i$の専有部の屋根面積の合計$A_{r,total,i}$ (m<sup>2</sup>)は次式により表される。

$$
A_{r,total,i} = \left\{
    \begin{array}{ll}
        A_{r,total,i,input} & (i \ne N) \\
        A_{f,total,i} & (i = N)
    \end{array}
\right.
$$

階$i$における専有部の屋根面積の合計は最上階とそれ以外の階に分けて求める。最上階以外は入力値とし、最上階は床面積の合計に等しいものとする。なお、最上階以外に屋根がある場合は、その階の主方位側かつ各住戸に均等に位置するものとする。

$N$: 建物の階数(階)  
$A_{r,h,total,i}$: $i$階における専有部(住戸部)の屋根面積の合計(m<sup>2</sup>)

In [14]:
def get_A_r_total( A_r_total_input, A_f_total, level, total_level_number):
    return A_r_total_input if level != total_level_number - 1 else A_f_total

### 1.2 外気に接する床の面積

階$i$における専有部の外気に接する床の面積の合計$A_{of,total,i}$ (m<sup>2</sup>) は次式により表される。  

$$
A_{of,total,i} = \left\{
    \begin{array}{ll}
        0 & (i = 1) \\
        A_{of,total,i,input} & (i \ne 1)
    \end{array}
\right.
$$

階$i$における専有部の外気に接する床の面積の合計は、1階(最下階)は0とし、1階以外は入力値とする。  
なお、最下階以外に外気に接する床がある場合は、その階の主方位側かつ各住戸に均等に位置するものとする。

$A_{of,total,i}$: 階$i$における専有部の外気に接する床の面積の合計(m<sup>2</sup>)

In [15]:
def get_A_of_total(A_of_total_input, level):
    return 0.0 if level == 0 else A_of_total_input

### 1.3 階高

階$i$の階高$H_i$(m)は次式により表される。

$$
H_i = \frac{H_{total}}{N}
$$

$H_i$: 階$i$の階高(m)  
$H_{total}$: 建物全体の高さ(m)  
$N$: 建物の階数  

In [46]:
def get_H( b ):
    H_total = b.height
    N       = b.levels
    return H_total / N

### 1.4 長辺および短辺の長さ

階$i$における専有部全体の平面形状は矩形を想定し、その長辺が主方位(主方位+180°)側にあるもとの仮定し、長辺と短辺の長さの合計は次式を連立して求めることとする。

$$ l_{long,total,i} \ge l_{short,total,i} $$
$$ l_{long,total,i} \times l_{short,total,i} = A_{f,total,i} $$
$$ 2 \times ( l_{long,total,i} + l_{short,total,i} ) = L_i $$

これらを連立すると、

$$ l_{long,total,i} = \frac{ \frac{L_i}{2} + \sqrt{ (\frac{L_i}{2})^2 - 4 \times A_{f,total,i} } }{2} $$

$$ l_{short,total,i} = \frac{ \frac{L_i}{2} - \sqrt{ (\frac{L_i}{2})^2 - 4 \times A_{f,total_i} } }{2} $$

となる。

$ A_{f,total,i} $: 階$i$における専有部の床面積の合計(m<sup>2</sup>)  
$ l_{long,total,i} $: 階$i$における専有部全体の長辺の長さの合計(m)  
$ l_{short,total,i} $: 階$i$における専有部全体の短辺の長さの合計(m)  
$ L_i$: 階$i$における専有部全体の外皮の周長(m)

In [17]:
def get_l_long_total(L, A_f_total):
    return ( (L/2) + math.sqrt( (L/2)**2 - 4 * A_f_total ) )/2

In [18]:
def get_l_short_total(L, A_f_total):
    return ( (L/2) - math.sqrt( (L/2)**2 - 4 * A_f_total ) )/2

### 1.5 窓面積

階$i$における主方位から時計回りに0°、90°、180°及び270°の方位の窓面積の合計は次式により算出する。なお、主方位を$D0$とし、方位の角度は上から見て時計回りに増加するものとし、$D90$で主方位+90度、$D180$で主方位+180度、$D270$で主方位+270度とする。（以下、同じ）。

$$
A_{win,total,D0,i} = A_{win,total,i} \times ( 1 - \frac{0.24}{n_i} ) \times 0.64
$$

$$
A_{win,total,D90,i} = A_{win,total,i} \times \frac{0.12}{n_i}
$$

$$
A_{win,total,D180,i} = A_{win,total,i} \times ( 1 - \frac{0.24}{n_i} ) \times 0.36
$$

$$
A_{win,total,D270,i} = A_{win,total,i} \times \frac{0.12}{n_i}
$$

$A_{win,total,i}$: 階$i$における窓面積の合計(m<sup>2</sup>)  
$A_{win,total,D0,i}$: 階$i$における主方位から時計回りに0°の方位の窓面積の合計(m<sup>2</sup>)  
$A_{win,total,D90,i}$: 階$i$における主方位から時計回りに90°の方位の窓面積の合計(m<sup>2</sup>)  
$A_{win,total,D180,i}$: 階$i$における主方位から時計回りに180°の方位の窓面積の合計(m<sup>2</sup>)  
$A_{win,total,D270,i}$: 階$i$における主方位から時計回りに270°の方位の窓面積の合計(m<sup>2</sup>)  
$n_i$: 階$i$における住戸数(戸)

In [19]:
def get_A_win_total( A_win_total, n ):
    return HorizontalOrientation(
        OrientationValue( A_win_total * (1-0.24/n) * 0.64, 'D0'   ),
        OrientationValue( A_win_total * 0.12/n,            'D90'  ),
        OrientationValue( A_win_total * (1-0.24/n) * 0.36, 'D180' ),
        OrientationValue( A_win_total * 0.12/n,            'D270' )
    )

### 2. 専有部における各階の各住戸の面積・長さ

### 2.1 屋根の面積

階$i$における住戸$j$の屋根面積$A_{r,i,j}$(m<sup>2</sup>)は次式により算出する。

$$
A_{r,i,j} = \frac{A_{r,total,i}}{n_i}
$$

$A_{r,i,j}$: 階$i$における住戸$j$の屋根面積 (m<sup>2</sup>)  
$A_{r,total,i}$: 階$i$における屋根面積の合計 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数 (戸)

In [20]:
def get_A_r( A_r_total, n ):
    return A_r_total / n

### 2.2 窓の面積

階$i$の住戸$j$における主方位から時計回りに0°、90°、180°及び270°の方位の窓面積$A_{win,D0,i,j}$, $A_{win,D90,i,j}$, $A_{win,D180,i,j}$, $A_{win,D270,i,j}$は、住戸位置ごとに次式により算出する。

$$
A_{win,D0,i,j} = \frac{ A_{win,total,D0,i} } { n_i }
$$

$$
A_{win,D90,i,j} = \left\{
    \begin{array}{ll}
        A_{win,total,D90,i} & (j=1) \\
        0 & (j \ne 1)
    \end{array}
\right.
$$

$$
A_{win,D180,i,j} = \frac{ A_{win,total,D180,i} } { n_i }
$$

$$
A_{win,D270,i,j} = \left\{
    \begin{array}{ll}
        A_{win,total,D270,i} & (j=n) \\
        0 & (j \ne n)
    \end{array}
\right.
$$

$A_{win,D0,i,j}$: 階$i$の住戸$j$における主方位から時計回りに0°の方位の窓面積(m<sup>2</sup>)  
$A_{win,D90,i,j}$: 階$i$の住戸$j$における主方位から時計回りに90°の方位の窓面積(m<sup>2</sup>)  
$A_{win,D180,i,j}$: 階$i$の住戸$j$における主方位から時計回りに180°の方位の窓面積(m<sup>2</sup>)  
$A_{win,D270,i,j}$: 階$i$の住戸$j$における主方位から時計回りに270°の方位の窓面積(m<sup>2</sup>)  
$A_{win,total,D0,i}$: 階$i$における主方位から時計回りに0°の方位の窓面積の合計(m<sup>2</sup>)  
$A_{win,total,D90,i}$: 階$i$における主方位から時計回りに90°の方位の窓面積の合計(m<sup>2</sup>)  
$A_{win,total,D180,i}$: 階$i$における主方位から時計回りに180°の方位の窓面積の合計(m<sup>2</sup>)   
$A_{win,total,D270,i}$: 階$i$における主方位から時計回りに270°の方位の窓面積の合計(m<sup>2</sup>)  

In [21]:
def get_A_win( A_win_total, housenumber, houseindex ):
    return HorizontalOrientation(
        OrientationValue( A_win_total.d0.v / housenumber,                             'D0'   ),
        OrientationValue( A_win_total.d90.v if houseindex == 0 else 0,                'D90'  ),
        OrientationValue( A_win_total.d180.v / housenumber,                           'D180' ),
        OrientationValue( A_win_total.d270.v if houseindex == housenumber - 1 else 0, 'D270' )
    )

### 2.3 玄関ドア面積

階$i$の住戸$j$における主方位から時計回りに0°、90°、180°及び270°の方位の各方位における玄関ドア面積$A_{door,D0,i,j}$, $A_{door,D90,i,j}$, $A_{door,D180,i,j}$, $A_{door,D270,i,j}$ (m<sup>2</sup>)は次式により算出する。

$$
A_{door,D0,i,j} = 0
$$

$$
A_{door,D90,i,j} = 0
$$

$$
A_{door,D180,i,j} = 1.6
$$

$$
A_{door,D270,i,j} = 0
$$

$A_{door,D0,i,j}$: 階$i$の住戸$j$における主方位から時計回りに0°の方位の玄関ドア面積(m<sup>2</sup>)  
$A_{door,D90,i,j}$: 階$i$の住戸$j$における主方位から時計回りに90°の方位の玄関ドア面積(m<sup>2</sup>)  
$A_{door,D180,i,j}$: 階$i$の住戸$j$における主方位から時計回りに180°の方位の玄関ドア面積(m<sup>2</sup>)  
$A_{door,D270,i,j}$: 階$i$の住戸$j$における主方位から時計回りに270°の方位の玄関ドア面積(m<sup>2</sup>)  

In [22]:
def get_A_door():
    return HorizontalOrientation(
        OrientationValue( 0.0, 'D0'   ),
        OrientationValue( 0.0, 'D90'  ),
        OrientationValue( 1.6, 'D180' ),
        OrientationValue( 0.0, 'D270' )
    ) 

### 2.4 外壁の面積

階$i$の住戸$j$における主方位から時計回りに0°、90°、180°及び270°の方位の各方位における外壁面積$A_{ow,D0,i,j}$, $A_{ow,D90,i,j}$, $A_{ow,D180,i,j}$, $A_{ow,D270,i,j}$は、住戸位置ごとに次式により算出する。なお、主方位+180°に玄関前を想定した窪み(幅1m、奥行き1m)を想定する。

$$
A_{ow,D0,i,j} = \frac{ l_{long,total,i} }{ n_i } \times H_i -  A_{win,D0,i,j} - A_{door,D0,i,j}
$$

$$
A_{ow,D90,i,j} = \left\{
    \begin{array}{ll}
        ( l_{short,total,i} + 1 ) \times H_i - A_{win,D90,i,j} - A_{door,D90,i,j} & (j=1) \\
        1 \times H_i - A_{win,D90,i} - A_{door,D90,i} & (j \ne 1)
    \end{array}
\right.
$$

$$
A_{ow,D180,i,j} = \frac{ l_{long,total,i} }{ n_i } \times H_i -  A_{win,D180,i,j} - A_{door,D180,i,j}
$$

$$
A_{ow,h,k,D270} = \left\{
    \begin{array}{ll}
        ( l_{short,total} + 1 ) \times H - A_{win,h,k,D270} - A_{door,h,D270} & (k=n) \\
        1 \times H - A_{win,h,k,D270} - A_{door,h,D270} & (k \ne n)
    \end{array}
\right.
$$

$A_{ow,D0,i,j}$: 階$i$の住戸$j$における主方位から時計回りに0°の方位の外壁面積(m<sup>2</sup>)  
$A_{ow,D90,i,j}$: 階$i$の住戸$j$における主方位から時計回りに90°の方位の外壁面積(m<sup>2</sup>)  
$A_{ow,D180,i,j}$: 階$i$の住戸$j$における主方位から時計回りに180°の方位の外壁面積(m<sup>2</sup>)  
$A_{ow,D270,i,j}$: 階$i$の住戸$j$における主方位から時計回りに270°の方位の外壁面積(m<sup>2</sup>)  
$l_{long,total,i}$: 階$i$における専有部全体の長辺の長さ (m)  
$l_{short,total,i}$: 階$i$における専有部全体の短辺の長さ (m)  
$n_i$: 階$i$における住戸数 (戸)  
$H_i$: 階$i$の階高 (m)  
$A_{win,D0,i,j}$: 階$i$の住戸$j$における主方位から時計回りに0°の方位の窓面積 (m<sup>2</sup>)  
$A_{win,D90,i,j}$: 階$i$の住戸$j$における主方位から時計回りに90°の方位の窓面積 (m<sup>2</sup>)  
$A_{win,D180,i,j}$: 階$i$の住戸$j$における主方位から時計回りに180°の方位の窓面積 (m<sup>2</sup>)  
$A_{win,D270,i,j}$: 階$i$の住戸$j$における主方位から時計回りに270°の方位の窓面積 (m<sup>2</sup>)  
$A_{door,D0,i,j}$: 階$i$の住戸$j$における主方位から時計回りに0°の方位の玄関ドア面積 (m<sup>2</sup>)  
$A_{door,D90,i,j}$: 階$i$の住戸$j$における主方位から時計回りに90°の方位の玄関ドア面積 (m<sup>2</sup>)  
$A_{door,D180,i,j}$: 階$i$の住戸$j$における主方位から時計回りに180°の方位の玄関ドア面積 (m<sup>2</sup>)  
$A_{door,D270,i,j}$: 階$i$の住戸$j$における主方位から時計回りに270°の方位の玄関ドア面積 (m<sup>2</sup>)  

In [23]:
def get_A_ow( l_long_total, l_short_total, H, A_win, A_door, houseNumber, houseIndex ):
    return HorizontalOrientation(
        OrientationValue( l_long_total / houseNumber * H                                           - A_win.d0.v   - A_door.d0.v,   'D0' ),
        OrientationValue( ( ( l_short_total + 1 ) * H if houseIndex == 0 else 1 * H )              - A_win.d90.v  - A_door.d90.v,  'D90' ),
        OrientationValue( l_long_total / houseNumber * H                                            - A_win.d180.v - A_door.d180.v, 'D180' ),
        OrientationValue( ( ( l_short_total + 1 ) * H if houseIndex == houseNumber - 1 else 1 * H ) - A_win.d270.v - A_door.d270.v, 'D270' )
    )

### 2.5 外気に接する床の面積

階$i$の住戸$j$における外気に接する床の面積$A_{of,i,j}$(m<sup>2</sup>)は次式により算出する。なお、最下階($i=1$)の場合は必ず$A_{of,total,i} = 0$のため、$A_{of,i,j}=0$である。

$$
A_{of,i,j} = \frac{ A_{of,total,i} }{ n_i }
$$

$A_{of,i,j}$: 階$i$の住戸$j$における外気に接する床の面積 (m<sup>2</sup>)  
$A_{of,total,i}$: 階$i$における専有部の外気に接する床の面積の合計 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数 (戸)

In [24]:
def get_A_of( A_of_total, n ):
    return A_of_total / n

### 2.6 界壁の面積

階$i$の住戸$j$の界壁面積$A_{iw,i,j}$(m<sup>2</sup>)は次式により算出する。

$ n_i = 1 $ (階$i$に1住戸しかない場合)の場合、

$$
A_{iw,i,j} = 0.0
$$

$ n_i > 2 $ の場合、

$$
A_{iw,i,j} = \left\{
    \begin{array}{ll}
        l_{short,total,i} \times H_i & ( j = 1 ) \\
        l_{short,total,i} \times H_i \times 2 & ( (j \ne 1) \  and \  (j \ne n_i) ) \\
        l_{short,total,i} \times H_i & ( j = n_i )
    \end{array}
\right.
$$

$n_i$: 階$i$における住戸数(戸)  
$A_{iw,i,j}$: 階$i$の住戸$j$における界壁の面積 (m<sup>2</sup>)  
$l_{short,total,i}$: 階$i$における専有部全体の短辺の長さ (m)  
$H_i$: 階$i$の階高 (m)

In [25]:
def get_A_iw( l_short_total, H, n, idx ):
    if idx == 0:
        return l_short_total * H
    elif idx == n - 1:
        return l_short_total * H
    else:
        return l_short_total * H * 2

### 2.7 界床の面積

階$i$の住戸$j$における界床の面積$A_{if,i,j}$ (m<sup>2</sup>) は、次式により算出する。

$$
A_{if,i,j} = \left\{
    \begin{array}{ll}
        0 & (i=1)\\
        \frac{ A_{f,total,i} }{ n_i } - A_{of,i,j} & ( i \ge 2 )
    \end{array}
\right.
$$

$A_{if,i,j}$: 階$i$の住戸$j$における界床面積 (m<sup>2</sup>)  
$A_{f,total,i}$: 階$i$の専有部の床面積の合計 (m<sup>2</sup>)  
$A_{of,i,j}$: 階$i$の住戸$j$における外気に接する床の面積 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数(戸)

In [26]:
def get_A_if(A_f_total, A_of, n, level):
    if level == 0:
        return 0.0
    else:
        return A_f_total / n - A_of

### 2.8 界天井の面積

階$i$の住戸$j$における界天井の面積$A_{ic,i,j}$ (m<sup>2</sup>) は、次式により算出する。

$$
A_{ic,i,j} = \left\{
    \begin{array}{ll}
        0 & (i=N)\\
        \frac{ A_{f,total,i} }{ n_i } - A_{r,i} & ( i < N )
    \end{array}
\right.
$$

$N$: 建物の階数(階)  
$A_{ic,i,j}$: 階$i$の住戸$j$における界天井の面積 (m<sup>2</sup>)  
$A_{f,total,i}$: 階$i$の専有部の床面積の合計 (m<sup>2</sup>)  
$A_{r,i}$: 階$i$の住戸$j$における外気に接する屋根の面積 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数(戸)

In [27]:
def get_A_ic( A_f_total, A_r, n, level, levelNumber ):
    if level == levelNumber - 1:
        return 0.0
    else:
        return A_f_total/n - A_r

### 2.9 床下に面する床面積

階$i$の住戸$j$における床下に面する床面積$A_{csf,i,j}$ (m<sup>2</sup>) は、次式により算出する。

$$
A_{csf,i,j} = \left\{
    \begin{array}{ll}
        \frac{ A_{f,total,i} }{ n_i } & (i=1)\\
        0 & ( i \ge 2 )
    \end{array}
\right.
$$

$A_{csf,i,j}$: 階$i$の住戸$j$における床下に面する床面積 (m<sup>2</sup>)  
$A_{f,total,i}$: 階$i$の専有部の床面積の合計 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数(戸)

In [28]:
def get_A_csf(A_f_total, n, level):
    if level > 2:
        return 0.0
    else:
        return A_f_total / n

### 3. 各住戸の熱橋長さ

![HeatBridge](HeatBridgeType.png)

各階における各住戸の構造熱橋部の長さ

| 種類(修正版)<br>(記号) | 種類<br>(記号) | 境界の数<br>(屋外/屋内/床下) | 熱橋の方向 | 主な発生場所 |
| --- | --- | --- | --- | --- |
| $ \psi_{1v,w-w} $  | $ \psi_{1v} $    | 3 / 1 / - | 鉛直 | 外壁同士の接合部 |
| $ \psi_{1h,w-r} $  | $ \psi_{1h,r} $  | 3 / 1 / - | 水平 | 外壁と屋根の接合部 |
| $ \psi_{1h,w-f} $  | $ \psi_{1h,f} $  | 3 / 1 / - | 水平 | 外壁と外気床の接合部 |
| $ \psi_{2v,w} $    | $ \psi_{2v} $    | 2 / 2 / - | 鉛直 | 外壁と間仕切壁の接合部（住戸長辺≧20mの場合は追加） |
| $ \psi_{2h,r} $    | $ \psi_{2h,r} $  | 2 / 2 / - | 水平 | 屋根と界壁の接合部（住戸長辺≧20mの場合は追加） |
| $ \psi_{2h,w} $    | $ \psi_{2h,f1} $ | 2 / 2 / - | 水平 | 外壁と界床の接合部 |
| $ \psi_{2h,f} $  | $ \psi_{2h,f2} $ | 2 / 2 / - | 水平 | 外気床と界壁の接合部 |
| $ \psi_{3v,w-w} $  | $ \psi_{3v} $    | 1 / 3 / - | 鉛直 | 外壁同士の接合部 |
| $ \psi_{3h,w-r} $  | $ \psi_{3h,r} $  | 1 / 3 / - | 水平 | 外壁と中間階屋根の接合部 |
| $ \psi_{3h,w-f} $  | $ \psi_{3h,f} $  | 1 / 3 / - | 水平 | 外壁と外気床の接合部 |
| $ \psi_{1h,w-uf} $ | $ \psi_{4h,uf} $ | 2 / 1 / 1 | 水平 | 外壁と最下階床の接合部 |
| $ \psi_{2h,uf} $   | $ \psi_{5h,uf} $ | - / 2 / 2 | 水平 | 界壁と最下階床の接合部 |


### 3.1 熱橋長さ$l_{\psi 1 v,w-w}$

階$i$における住戸$j$の$\psi_{1v,i,j}$の熱橋長さは、住戸位置ごとに下式により算出する。なお、方位+180に玄関前を想定した窪み(幅1m、奥行き1m)を想定する（以下、同じ）。

$$
l_{\psi 1v,D0-D90,i,j} = \left \{
    \begin{array}{ll}
        H_i & ( j = 1 ) \\
        0 & ( j \ne 1 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 1v,D90-D180,i,j} = \left \{
    \begin{array}{ll}
        H_i \times 2 & ( j = 1 ) \\
        H_i & ( j \ne 1 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 1v,D180-D270,i,j} = \left \{
    \begin{array}{ll}
        H_i \times 2 & ( j = n ) \\
        H_i & ( j \ne n ) \\
    \end{array}
\right.
$$

$$
l_{\psi 1v,D270-D0,i,j} = \left \{
    \begin{array}{ll}
        H_i & ( j = n ) \\
        0 & ( j \ne n ) \\
    \end{array}
\right.
$$

In [29]:
def get_l_psi_1v_ww( H, n, idx ):
    return [
        OrientationValueHB( H     if idx == 0     else 0.0, 'D0',   'D90'  ),
        OrientationValueHB( H * 2 if idx == 0     else H,   'D90',  'D180' ),
        OrientationValueHB( H * 2 if idx == n - 1 else H,   'D180', 'D270' ),
        OrientationValueHB( H     if idx == n - 1 else 0.0, 'D270', 'D0'   )
    ]

### 3.2 熱橋長さ$l_{\psi 1h,w-r}$

#### i) 最上階以外($i \ne N$)の場合でかつ屋根がある($A_{r,total,i} \gt 0$)場合

$$
l_{\psi 1h,w-r,top-D0,i,j} = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 1h,w-r,top-D90,i,j} = \left \{
    \begin{array}{ll}
        \frac{ A_{r,total,i} }{ l_{long,total,i} } & ( j = 1 ) \\
        0 & ( j \ne 1 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 1h,w-r,top-D180,i,j} = 0
$$

$$
l_{\psi 1h,w-r,top-D270,i,j} = \left \{
    \begin{array}{ll}
        \frac{ A_{r,total,i} }{ l_{long,total,i} } & ( j = n_i ) \\
        0 & ( j \ne n_i ) \\
    \end{array}
\right.
$$

#### ii) 最上階以外($i \ne N_i$)の場合でかつ屋根がない($A_{r,total,i} = 0$)場合

$$
l_{\psi 1h,w-r,top-D0,i,j} = 0
$$

$$
l_{\psi 1h,w-r,top-D90,i,j} = 0
$$

$$
l_{\psi 1h,w-r,top-D180,i,j} = 0
$$

$$
l_{\psi 1h,w-r,top-D270,i,j} = 0
$$

#### iii) 最上階($i = N$)の場合

$$
l_{\psi 1h,w-r,top-D0,i,j} = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 1h,w-r,top-D90,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} + 1 & ( j = 1 ) \\
        1 & ( j \ne 1 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 1h,w-r,top-D180,i,j} = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 1h,w-r,top-D270,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} + 1 & ( j = n ) \\
        1 & ( j \ne n ) \\
    \end{array}
\right.
$$

In [30]:
def get_l_psi_1h_wr( l_long_total, l_short_total, A_r_total, n, idx, level, levelNumber ):
    if level == levelNumber:
        return [
            OrientationValueHB( l_long_total / n,                           'top', 'D0'   ),
            OrientationValueHB( l_short_total + 1 if idx == 0 else 1.0,     'top', 'D90'  ),
            OrientationValueHB( l_long_total / n,                           'top', 'D180' ),
            OrientationValueHB( l_short_total + 1 if idx == n - 1 else 1.0, 'top', 'D270' ),
        ]
    else:
        if A_r_total == 0:
            return [
                OrientationValueHB( 0.0, 'top', 'D0'   ),
                OrientationValueHB( 0.0, 'top', 'D90'  ),
                OrientationValueHB( 0.0, 'top', 'D180' ),
                OrientationValueHB( 0.0, 'top', 'D270' ),
            ]
        else:
            return[
                OrientationValueHB( l_long_total / n,                                  'top', 'D0'   ),
                OrientationValueHB( A_r_total / l_long_total if idx == 0 else 0.0,     'top', 'D90'  ),
                OrientationValueHB( 0.0,                                               'top', 'D180' ),
                OrientationValueHB( A_r_total / l_long_total if idx == n - 1 else 0.0, 'top', 'D270' ),
            ]        

### 3.3 熱橋長さ$l_{\psi 1h,w-f}$

#### i) 外気床がある($A_{of,total,i} \gt 0$)場合

$$
l_{\psi 1h,w-f,bottom-D0,i,j} = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 1h,w-f,bottom-D90} = \left \{
    \begin{array}{ll}
        \frac{ A_{of,total,i} }{ l_{long,total,i} } & ( j = 1 ) \\
        0 & ( j \ne 1 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 1h,w-f,bottom-D180,i,j} = 0
$$

$$
l_{\psi 1h,w-f,bottom-D270,i,j} = \left \{
    \begin{array}{ll}
        \frac{ A_{of,total,i} }{ l_{long,total,i} } & ( j = n ) \\
        0 & ( j \ne n ) \\
    \end{array}
\right.
$$

#### ii) 外気床がない($A_{of,total,i} = 0$)場合

$$
l_{\psi 1h,w-f,bottom-D0,i,j} = 0
$$

$$
l_{\psi 1h,w-f,bottom-D90,i,j} = 0
$$

$$
l_{\psi 1h,w-f,bottom-D180,i,j} = 0
$$

$$
l_{\psi 1h,w-f,bottom-D270,i,j} = 0
$$

In [31]:
def get_l_psi_1h_wf( l_long_total, A_of_total, n, idx ):
    if A_of_total > 0:
        return [
            OrientationValueHB( l_long_total / n,                                   'bottom', 'D0'   ),
            OrientationValueHB( A_of_total / l_long_total if idx == 0 else 0.0,     'bottom', 'D90'  ),
            OrientationValueHB( 0.0,                                                'bottom', 'D180' ),
            OrientationValueHB( A_of_total / l_long_total if idx == n - 1 else 0.0, 'bottom', 'D270' ),
        ]
    else:
        return [
            OrientationValueHB( 0.0, 'bottom', 'D0'   ),
            OrientationValueHB( 0.0, 'bottom', 'D90'  ),
            OrientationValueHB( 0.0, 'bottom', 'D180' ),
            OrientationValueHB( 0.0, 'bottom', 'D270' ),
        ]

### 3.4 熱橋長さ$l_{\psi 2v,w}$

$$
l_{\psi 2v,w,D0,i,j} = \left \{
    \begin{array}{ll}
        0 & ( n_i = 1 ) \\
        H_i & ( n_i \gt 1 \  and \  j = 1 ) \\
        H_i & ( n_i \gt 1 \  and \  j = n ) \\
        H_i \times 2 & ( n_i \gt 1 , \  j \ne 1 \  and \  j \ne n ) \\
    \end{array}
\right.
$$

$$
l_{\psi 2v,w,D90,i,j} = \left \{
    \begin{array}{ll}
        H_i \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( j = 1 ) \\
        0 & ( j \ne 1 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 2v,w,D180,i,j} = \left \{
    \begin{array}{ll}
        0 & ( n_i = 1 ) \\
        H_i & ( n_i \gt 1 \  and \  j = 1 ) \\
        H_i & ( n_i \gt 1 \  and \  j = n ) \\
        H_i \times 2 & ( n_i \gt 1 , \  j \ne 1 \  and \  j \ne n ) \\
    \end{array}
\right.
$$

$$
l_{\psi 2v,w,D270,i,j} = \left \{
    \begin{array}{ll}
        H_i \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( j = n ) \\
        0 & ( j \ne n ) \\
    \end{array}
\right.
$$

In [32]:
def get_l_psi_2v_w( l_short_total, H, n, idx ):
    return [
        OrientationValueHB( 0.0 if n == 1 else ( H if idx == 0 or idx == n - 1 else H * 2 ), 'D0',   'D0'   ),
        OrientationValueHB( H * max( l_short_total - 10, 0.0 ) / 10 if idx == 0 else 0.0,    'D90',  'D90'  ),
        OrientationValueHB( 0.0 if n == 1 else ( H if idx == 0 or idx == n - 1 else H * 2 ), 'D180', 'D180' ),
        OrientationValueHB( H * max( l_short_total - 10, 0.0 ) / 10 if idx == n-1 else 0.0,  'D270', 'D270' ),
    ]

### 3.5 熱橋長さ$l_{\psi 2h,r}$

#### i) 最上階以外($i \ne N$)の場合でかつ屋根がある($A_{r,total,i} \gt 0$)場合

$$
l_{\psi 2h,r,top,i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{r,total,i} }{ l_{long,total,i} } - 10, \  0 \right) }{ 10 } & ( n_i = 1 ) \\
        \frac{ A_{r,total,i} }{ l_{long,total,i} } + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{r,total,i} }{ l_{long,total,i} } - 10, \  0 \right) }{ 10 } & ( n_i > 1 \  and \  j = 1 ) \\
        \frac{ A_{r,total,i} }{ l_{long,total,i} } + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{r,total,i} }{ l_{long,total,i} } - 10, \  0 \right) }{ 10 } & ( n_i > 1 \  and \  j = n ) \\
        \frac{ A_{r,total,i} }{ l_{long,total,i} } \times 2 + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{r,total,i} }{ l_{long,total,i} } - 10, \  0 \right) }{ 10 } & ( n_i > 1 , \  j \ne 1 \  and \ j \ne n ) \\
    \end{array}
\right.
$$

#### ii) 最上階以外($i \ne N$)の場合でかつ屋根がない($A_{r,total,i} = 0$)場合

$$
l_{\psi 2h,r,top,i,j} = 0
$$

#### iii) 最上階($i = N$)の場合

$$
l_{\psi 2h,r,top,i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i = 1 ) \\
        l_{short,total,i} + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right)}{ 10 } & ( n_i > 1 \  and \  j = 1 ) \\
        l_{short,total,i} + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i > 1 \  and \  j = n ) \\
        l_{short,total,i} \times 2 + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i > 1 , \  j \ne 1 \  and \ k \ne n ) \\
    \end{array}
\right.
$$

In [33]:
def get_l_psi_2h_r( A_r_total, l_long_total, l_short_total, n, idx, level, levelNumber ):
    if level == levelNumber:
        return [
            OrientationValueHB( 0.0 if n == 1 else ( l_short_total if idx == 0 or idx == n - 1 else l_short_total * 2 )
                                 + l_long_total / n * max( l_short_total - 10.0, 0.0 ) / 10.0, 'top', 'top' )
        ]
    else:
        if A_r_total == 0.0:
            return [ OrientationValueHB( 0.0, 'top', 'top' ) ]
        else:
            return [
                OrientationValueHB(
                0.0 if n == 1 else ( A_r_total / l_long_total if idx == 0 or idx == n - 1 else A_r_total / l_long_total * 2 )
                + l_long_total / n * max( A_r_total / l_long_total - 10.0, 0.0 ) / 10.0, 'top', 'top' )
            ]

### 3.6 熱橋長さ$l_{\psi 2h,w}$

#### i) 最下階($i = 1$)の場合

$$
l_{\psi 2 h,w,D0, i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } & ( A_{of,total,i+1} = 0 ) \\
        0 & ( A_{of,total,i+1} > 0 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 2 h, w, D90,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} - \frac{ A_{r,total,i} }{ l_{long,total,i} } + 1 & ( j = 0 ) \\
        1 & ( j \ne 0 ) \\
    \end{array}
\right.
$$

$$
l_{ \psi 2 h, w, D180,i,j } = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 2 h, w, D270,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} - \frac{ A_{r,total,i} }{ l_{long,total,i} } + 1 & ( j = n_i ) \\
        1 & ( j \ne n_i ) \\
    \end{array}
\right.
$$

#### ii) 最上階($i = N$)の場合

$$
l_{\psi 2 h,w,D0, i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } & ( A_{r,total,i-1} = 0 ) \\
        0 & ( A_{r,total,i-1} > 0 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 2 h, w, D90,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} - \frac{ A_{of,total,i} }{ l_{long,total,i} } + 1 & ( j = 0 ) \\
        1 & ( j \ne 0 ) \\
    \end{array}
\right.
$$

$$
l_{ \psi 2 h, w, D180,i,j } = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 2 h, w, D270,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} - \frac{ A_{of,total,i} }{ l_{long,total,i} } + 1 & ( j = n_i ) \\
        1 & ( j \ne n_i ) \\
    \end{array}
\right.
$$

#### iii) 中間階($i \ne 0 \  and \  i \ne N$)の場合

$$
l_{\psi 2 h,w,D0, i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } \times 2 & ( ( A_{of,total,i+1} = 0 ) \  and \  ( A_{r,total,i-1} = 0 ) ) \\
        \frac{ l_{long,total,i} }{ n_i } & ( ( A_{of,total,i+1} > 0 ) \  and \  ( A_{r,total,i-1} = 0 ) ) \\
        \frac{ l_{long,total,i} }{ n_i } & ( ( A_{of,total,i+1} = 0 ) \  and \  ( A_{r,total,i-1} > 0 ) ) \\
        0 & ( ( A_{of,total,i+1} > 0 ) \  and \  ( A_{r,total,i-1} > 0 ) ) \\
    \end{array}
\right.
$$

$$
l_{\psi 2 h, w, D90,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} \times 2 - \frac{ A_{r,total,i} }{ l_{long,total,i} } - \frac{ A_{of,total,i} }{ l_{long,total,i} } + 2 & ( j = 0 ) \\
        2 & ( j \ne 0 ) \\
    \end{array}
\right.
$$

$$
l_{ \psi 2 h, w, D180,i,j } = \frac{ l_{long,total,i} }{ n_i } \times 2
$$

$$
l_{\psi 2 h, w, D270,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} \times 2 - \frac{ A_{r,total,i} }{ l_{long,total,i} } - \frac{ A_{of,total,i} }{ l_{long,total,i} } + 2 & ( j = n_i ) \\
        2 & ( j \ne n_i ) \\
    \end{array}
\right.
$$

In [34]:
def get_l_psi_2h_w( A_r_total, A_r_total_mns, A_of_total, A_of_total_pls, l_long_total, l_short_total, n, idx, level, levelNumber ):
    if level == 0:
        return [
            OrientationValueHB( l_long_total / n if A_of_total_pls == 0.0 else 0.0,                     'D0',   'D0'    ),
            OrientationValueHB( l_short_total - A_r_total / l_long_total + 1.0 if idx == 0 else 1.0,    'D90',  'D90'   ),
            OrientationValueHB( l_long_total / n,                                                        'D180', 'D180' ),
            OrientationValueHB( l_short_total - A_r_total / l_long_total + 1.0 if idx == n - 1 else 1.0, 'D270', 'D270' )
        ]
    elif level == levelNumber:
        return [
            OrientationValueHB( l_long_total / n if A_r_total_mns == 0.0 else 0.0,                        'D0',   'D0'   ),
            OrientationValueHB( l_short_total - A_of_total / l_long_total + 1.0 if idx == 0 else 1.0,     'D90',  'D90'  ),
            OrientationValueHB( l_long_total / n,                                                         'D180', 'D180' ),
            OrientationValueHB( l_short_total - A_of_total / l_long_total + 1.0 if idx == n - 1 else 1.0, 'D270', 'D270' )
        ]
    else:
        return [
            OrientationValueHB( l_long_total / n * 2 if A_of_total_pls == 0.0 and A_r_total_mns == 0.0
                               else ( 0.0 if A_of_total_pls > 0.0 and A_r_total_mns > 0.0 else l_long_total / n ), 'D0', 'D0' ),
            OrientationValueHB( l_short_total * 2 - A_r_total / l_long_total - A_of_total / l_long_total + 2.0 if idx == 0
                               else 2.0, 'D90', 'D90' ),
            OrientationValueHB( l_long_total / n * 2, 'D180', 'D180' ),
            OrientationValueHB( l_short_total * 2 - A_r_total / l_long_total - A_of_total / l_long_total + 2.0 if idx == n - 1
                               else 2.0, 'D270', 'D270' )
        ]

### 3.7 熱橋長さ$l_{\psi 2h,f}$

#### i) 外気床がある($A_{of,total,i} \gt 0$)場合

$$
l_{\psi 2h,f,bottom,i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{of,total,i} }{ l_{long,total,i} } - 10.0, \  0.0 \right) }{ 10 } & ( n_i = 1 ) \\
        \frac{ A_{of,total,i} }{ l_{long,total,i} } + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{of,total,i} }{ l_{long,total,i} } - 10.0, \  0.0 \right) }{ 10 } & ( n_i > 1 \  and \  j = 1 ) \\
        \frac{ A_{of,total,i} }{ l_{long,total,i} } + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{of,total,i} }{ l_{long,total,i} } - 10.0, \  0.0 \right) }{ 10 } & ( n_i > 1 \  and \  j = n ) \\
        \frac{ A_{of,total,i} }{ l_{long,total,i} } \times 2 + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{of,total,i} }{ l_{long,total,i} } - 10.0, \  0.0 \right) }{ 10 } & ( n_i > 1 , \  j \ne 1 \  and \ j \ne n ) \\
    \end{array}
\right.
$$

#### ii) 外気床がない($A_{of,total,i} = 0$)場合

$$
l_{\psi 2h,f,bottom,i,j} = 0
$$

In [35]:
def get_l_psi_2h_f( A_of_total, l_long_total, n, idx ):
    if A_of_total > 0:
        if n == 1:
            return [ OrientationValueHB( l_long_total / n * max( A_of_total / l_long_total - 10.0, 0.0 ) / 10.0, 'bottom', 'bottom' ) ]
        else:
            if idx == 0 or idx == n - 1:
                return [ OrientationValueHB( A_of_total / l_long_total
                                          + l_long_total / n * max( A_of_total / l_long_total - 10.0, 0.0 ) / 10.0, 'bottom', 'bottom' ) ]
            else:
                return [ OrientationValueHB( A_of_total / l_long_total * 2
                                          + l_long_total / n * max( A_of_total / l_long_total - 10.0, 0.0 ) / 10.0, 'bottom', 'bottom' ) ]
    else:
        return [ OrientationValueHB( 0.0, 'bottom', 'bottom' ) ]

### 3.8 熱橋長さ$l_{\psi 3v,w-w}$

$$
l_{\psi 3 v, w-w, D90-D180,i,j} = H_i \times 2
$$

$$
l_{\psi 3 v, w-w, D180-D270,i,j} = H_i \times 2
$$

In [36]:
def get_l_psi_3v_ww( H ):
    return [ OrientationValueHB( H * 2, 'D90',  'D180' ),
             OrientationValueHB( H * 2, 'D180', 'D270' ) ]

### 3.9 熱橋長さ$l_{\psi 3h,w-r}$

#### i) 最下階($i = 1$)の場合

$$
l_{\psi 3 h, w-r, top-D0,i,j} = \left \{
    \begin{array}{ll}
        0 & (A_{r,total,i}=0) \\
        \frac{ l_{long,total,i} }{ n_i } & (A_{r,total,i}>0) \\
    \end{array}
\right.
$$

#### ii) 最上階($i=N$)の場合

$$
l_{\psi 3 h, w-r, top-D0,i,j} = \left \{
    \begin{array}{ll}
        0 & (A_{r,total,i-1}=0) \\
        \frac{ l_{long,total,i} }{ n_i } & (A_{r,total,i-1}>0) \\
    \end{array}
\right.
$$

#### iii) 中間階($(i \ne 1) \  and \  (i \ne N)$)の場合

$$
l_{\psi 3 h, w-r, top-D0,i,j} = \left \{
    \begin{array}{ll}
        0 & ((A_{r,total,i}=0)\  and \  (A_{r,total,i-1}=0)) \\
        \frac{ l_{long,total,i} }{ n_i } & ((A_{r,total,i}>0)\  and \  (A_{r,total,i-1}=0)) \\
        \frac{ l_{long,total,i} }{ n_i } & ((A_{r,total,i}=0)\  and \  (A_{r,total,i-1}>0)) \\
        \frac{ l_{long,total,i} }{ n_i } \times 2 & ((A_{r,total,i}>0)\  and \  (A_{r,total,i-1}>0)) \\
    \end{array}
\right.
$$

In [37]:
def get_l_psi_3h_wr( l_long_total, A_r_total, A_r_total_mns, n, level, levelNumber ):
    if level == 0:
        return [ OrientationValueHB( 0.0 if A_r_total == 0.0 else l_long_total / n, 'top', 'D0' ) ]
    elif level == levelNumber:
        return [ OrientationValueHB( 0.0 if A_r_total_mns == 0.0 else l_long_total / n, 'top', 'D0' ) ]
    else:
        return [ OrientationValueHB( 0.0 if A_r_total == 0.0 and A_r_total_mns == 0.0 
                                   else ( l_long_total / n * 2 if A_r_total > 0.0 and A_r_total_mns > 0.0 else l_long_total / n ),
                                   'top', 'D0' ) ]

### 3.10 熱橋長さ$l_{\psi 3h,w-f}$

#### i) 最上階($i=N$)の場合

$$
l_{\psi 3 h, w-f, bottom-D0,i,j} = \left \{
    \begin{array}{ll}
        0 & (A_{of,total,i}=0) \\
        \frac{ l_{long,total,i} }{ n_i } & (A_{of,total,i}>0) \\
    \end{array}
\right.
$$

#### ii) 最上階以外($i \ne N$)の場合

$$
l_{\psi 3 h, w-f, bottom-D0,i,j} = \left \{
    \begin{array}{ll}
        0 & ((A_{of,total,i}=0)\  and \  (A_{of,total,i+1}=0)) \\
        \frac{ l_{long,total,i} }{ n_i } & ((A_{of,total,i}>0)\  and \  (A_{of,total,i+1}=0)) \\
        \frac{ l_{long,total,i} }{ n_i } & ((A_{of,total,i}=0)\  and \  (A_{of,total,i+1}>0)) \\
        \frac{ l_{long,total,i} }{ n_i } \times 2 & ((A_{of,total,i}>0)\  and \  (A_{of,total,i+1}>0)) \\
    \end{array}
\right.
$$

In [38]:
def get_l_psi_3h_wf( A_of_total, A_of_total_pls, level, levelNumber ):
    if level == levelNumber:
        return [ OrientationValueHB( 0.0 if A_of_total == 0.0 else l_long_total / n, 'bottom', 'D0' ) ]
    else:
        return [ OrientationValueHB( 0.0 if A_of_total == 0.0 and A_of_total_pls == 0.0
                                   else ( l_long_total / n * 2 if A_of_total > 0.0 and A_of_total_pls > 0.0 else l_long_total / n ),
                                   'bottom', 'D0' ) ]

### 3.11 熱橋長さ$l_{\psi 1h,w-uf}$

$$
l_{\psi 1 h, w-uf, bottom-D0,i,j} = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 1 h, w-uf,bottom-D90,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} + 1 & (j=1) \\
        1 & (j \ne 1) \\
    \end{array}
\right.
$$

$$
l_{\psi 1 h, w-uf, bottom-D180,i,j} = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 1 h, w-uf,bottom-D270,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} + 1 & (j=n_i) \\
        1 & (j \ne n_i) \\
    \end{array}
\right.
$$

In [72]:
def get_l_psi_1h_wuf( l_long_total, l_short_total, idx, n ):
    return [
        OrientationValueHB( l_long_total / n, 'bottom', 'D0' ),
        OrientationValueHB( l_short_total + 1.0 if idx == 0 else 1.0, 'bottom', 'D90' ),
        OrientationValueHB( l_long_total / n, 'bottom', 'D180' ),
        OrientationValueHB( l_short_total + 1.0 if idx == 0 else n - 1, 'bottom', 'D270' ),
    ]

### 3.12 熱橋長さ$l_{\psi 2h,uf}$

#### i) 最下階($i=1$)の場合

$$
l_{\psi 2 h,uf,bottom,i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i = 1 ) \\
        l_{short,total,i} + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i > 1 \  and \  j = 1 ) \\
        l_{short,total,i} + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i > 1 \  and \  j = n_i ) \\
        l_{short,total,i} \times 2 + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i > 1 , \  j \ne 1 \  and \ j \ne n_i ) \\
    \end{array}
\right.
$$

#### ii) 最下階($i \ne 1$)の場合

$$
l_{\psi 2 h,uf,bottom,i,j} = 0
$$

In [40]:
def get_l_psi_2h_uf( l_long_total, l_short_total, n, idx, level ):
    if level == 0:
        if n == 1:
            return [
                OrientationValueHB( l_long_total / n * max( l_short_total - 10.0, 0.0 ) / 10.0, 'bottom', 'bottom' )
            ]
        else:
            if idx != 0 and idx != n - 1:
                return [
                    OrientationValueHB( l_short_total *  2 + l_long_total / n * max( l_short_total - 10.0, 0.0 ) / 10.0, 'bottom', 'bottom')
                ]
            else:
                return [
                    OrientationValueHB( l_short_total + l_long_total / n * max( l_short_total - 10.0, 0.0 ) / 10.0, 'bottom', 'bottom')
                ]
    else:
        return [ OrientationValueHB( 0.0, 'bottom', 'bottom' ) ]

## クラス

In [52]:
class Building():
    
    def __init__( self, d ):
        self.region      = d['General']['Region']
        self.structure   = d['General']['Structure']
        self.levels      = d['General']['LevelsNumber']
        self.height      = d['General']['Height']
        self.direction   = d['General']['OpeningMainDirection']
        self.privatearea = PrivateArea( d )
    
    def calcAL( self ):
        self.privatearea.calcAL( self )
    
    def __str__( self ):
        s  = '建物情報' + '\n'
        s += '      地域  : ' + str(self.region) + '\n'
        s += '      構造  : ' + str(self.structure) + '\n'
        s += '      階数  : ' + str(self.levels) + '\n'
        s += '      高さ  : ' + str(self.height) + '\n'
        s += '      主方位: ' + str(self.direction)
        return s 

In [64]:
class PrivateArea():
    
    def __init__( self, d ):
        self.wallUValue        = d['PrivateArea']['Wall']['UValue']
        self.wallInsPos        = d['PrivateArea']['Wall']['InsPos']
        self.wallHBSpec        = d['PrivateArea']['Wall']['HBSpec']
        self.roofUValue        = d['PrivateArea']['Roof']['UValue']
        self.roofInsPos        = d['PrivateArea']['Roof']['InsPos']
        self.roofHBSpec        = d['PrivateArea']['Roof']['HBSpec']
        self.floorUValue       = d['PrivateArea']['Floor']['UValue']
        self.floorInsPos       = d['PrivateArea']['Floor']['InsPos']
        self.floorHBSpec       = d['PrivateArea']['Floor']['HBSpec']
        self.insideWallUValue  = d['PrivateArea']['InsideWallUValue']
        self.insideFloorUValue = d['PrivateArea']['InsideFloorUValue']
        self.windowUValue      = d['PrivateArea']['WindowUValue']
        self.windowEtaValue    = d['PrivateArea']['WindowEtaValue']
        self.floors            = [ PrivateFloor( i, d ) for i in range(d['General']['LevelsNumber']) ]
    
    def calcAL( self, building ):
        for f in self.floors:
            f.calcAL( building, self )
    
    def __str__( self ):
        s  = '   専有部' + '\n'
        s += '         壁  : ' + 'U値:' + str(self.wallUValue)  + ', 断熱材位置:' + str(self.wallInsPos)  + ', 熱橋タイプ:' + str(self.wallHBSpec)  + '\n'
        s += '         屋根: ' + 'U値:' + str(self.roofUValue)  + ', 断熱材位置:' + str(self.roofInsPos)  + ', 熱橋タイプ:' + str(self.roofHBSpec)  + '\n'
        s += '         床  : ' + 'U値:' + str(self.floorUValue) + ', 断熱材位置:' + str(self.floorInsPos) + ', 熱橋タイプ:' + str(self.floorHBSpec) + '\n'
        s += '         界壁: ' + 'U値:' + str(self.insideWallUValue)  + '\n'
        s += '         界床: ' + 'U値:' + str(self.insideFloorUValue) + '\n'
        s += '         窓  : ' + 'U値:' + str(self.windowUValue) + ', η値:' + str(self.windowEtaValue)
        return s

In [65]:
class PrivateFloor():
    
    def __init__( self, level, d ):
        self.level = level
        self.houseNumber       = d['PrivateArea']['HouseNumber'][level]
        self.floorArea         = d['PrivateArea']['FloorArea'][level]
        self.periLengthOutside = d['PrivateArea']['PeriLengthOutside'][level]
        self.windowArea        = d['PrivateArea']['WindowArea'][level]
        self.roofArea          = get_A_r_total( d['PrivateArea']['RoofArea'][level],
                                                d['PrivateArea']['FloorArea'][level],
                                                level, d['General']['LevelsNumber'] )
        self.outsideFloorArea  = get_A_of_total( d['PrivateArea']['OutsideFloorArea'][level], level )
        self.houses            = [ House( i, d ) for i in range( self.houseNumber ) ] 
        
    def calcAL( self, building, privatearea ):
        
        # 階高
        self.height = get_H( building )
        
        # 長辺・短辺の長さ
        self.l_long_total  = get_l_long_total( self.periLengthOutside, self.floorArea )
        self.l_short_total = get_l_short_total( self.periLengthOutside, self.floorArea )
        
        # 窓面積の合計
        self.windowsArea = get_A_win_total( self.windowArea, self.houseNumber )
        
        for h in self.houses:
            h.calcAL( building, privatearea, self )
        
    def __str__( self ):
        s = '      階' + str(self.level) + '\n'
        s += '            住戸数    : ' + str(self.houseNumber) + '\n'
        s += '            床面積    : ' + str(self.floorArea) + '\n'
        s += '            外周長    : ' + str(self.periLengthOutside) + '\n'
        s += '            窓面積    : ' + str(self.windowArea) + '\n'
        s += '            屋根面積  : ' + str(self.roofArea) + '\n'
        s += '            外気床面積: ' + str(self.outsideFloorArea) + '\n'
        s += '            階高      : ' + str(self.height) + '\n'
        s += '            長辺長さ  : ' + str(self.l_long_total) + '\n'
        s += '            短辺長さ  : ' + str(self.l_short_total) + '\n'
        s += '            窓面積    : ' + '\n'
        for windowArea in self.windowsArea.getMembers():
            s += '               ' + str(windowArea) + '\n'
        return s

In [73]:
class House():
    
    def __init__( self, number, d ):
        self.number = number
    
    def calcAL( self, building, privatearea, floor ):
        
        # 屋根面積
        self.roofArea = get_A_r( floor.roofArea, floor.houseNumber )
        
        # 窓面積
        self.windowsArea = get_A_win( floor.windowsArea, floor.houseNumber, self.number )
        
        # 玄関ドア面積
        self.doorsArea = get_A_door()
        
        # 外壁面積
        self.outsideWallsArea = get_A_ow( floor.l_long_total, floor.l_short_total, floor.height,
                                         self.windowsArea, self.doorsArea, floor.houseNumber, self.number )
        
        # 外気に面する床面積
        self.outsideFloorArea = get_A_of( floor.outsideFloorArea, floor.houseNumber )

        # 界壁面積
        self.insideWallArea = get_A_iw( floor.l_short_total, floor.height, floor.houseNumber, self.number )
        
        # 界床面積
        self.insideFloorArea = get_A_if( floor.floorArea, self.outsideFloorArea, floor.houseNumber, floor.level )

        # 界天井面積
        self.insideCeilingArea = get_A_ic( floor.floorArea, self.roofArea, floor.houseNumber, floor.level, building.levels )
        
        # 床下に面する床面積
        self.crawlSpaceFloorArea = get_A_csf( floor.floorArea, floor.houseNumber, floor.level )
        
        # 熱橋長さ
        self.l_psi_1v_ww = get_l_psi_1v_ww( floor.height, floor.houseNumber, self.number )
        self.l_psi_1h_wr = get_l_psi_1h_wr( floor.l_long_total, floor.l_short_total, floor.roofArea, floor.houseNumber, self.number,
                                           floor.level, building.levels )
        self.l_psi_1h_wf = get_l_psi_1h_wf( floor.l_long_total, floor.outsideFloorArea, floor.houseNumber, self.number )
        self.l_psi_2v_w  = get_l_psi_2v_w( floor.l_short_total, floor.height, floor.houseNumber, self.number )
        
        # 書きかけ
        self.l_psi_2h_r  = get_l_psi_2h_r( floor.roofArea, floor.l_long_total, floor.l_short_total, floor.houseNumber,
                                          self.number, floor.level, building.levels )
        A_of_total_pls = 0.0 if floor.level == building.levels - 1 else building.privatearea.floors[floor.level+1].outsideFloorArea
        A_r_total_mns  = 0.0 if floor.level == 0 else building.privatearea.floors[floor.level-1].roofArea
        self.l_psi_2h_w   = get_l_psi_2h_w( floor.roofArea, A_r_total_mns, floor.outsideFloorArea,A_of_total_pls, floor.l_long_total,
                                            floor.l_short_total, floor.houseNumber, self.number, floor.level, building.levels )
        self.l_psi_2h_f   = get_l_psi_2h_f( floor.outsideFloorArea, floor.l_long_total, floor.houseNumber, self.number )
        self.l_psi_3v_ww  = get_l_psi_3v_ww( floor.height )
        self.l_psi_3h_wr  = get_l_psi_3h_wr( floor.l_long_total, floor.roofArea, A_r_total_mns, floor.houseNumber, floor.level, building.levels )
        self.l_psi_3h_wf  = get_l_psi_3h_wf( floor.outsideFloorArea, A_of_total_pls, floor.level, building.levels )
        self.l_psi_1h_wuf = get_l_psi_1h_wuf( floor.l_long_total, floor.l_short_total, self.number, floor.houseNumber )
        self.l_psi_2h_uf  = get_l_psi_2h_uf( floor.l_long_total, floor.l_short_total, floor.houseNumber, self.number, floor.level )
            
    def __str__( self ):
        s = '         住戸:' + str(self.number) + '\n'
        s += '               屋根面積      : ' + str(self.roofArea) + '\n'
        s += '               窓面積        : ' + '\n'
        for windowArea in self.windowsArea.getMembers():
            s += '                  ' + str(windowArea) + '\n'
        s += '               玄関ドア面積  : ' + '\n'
        for doorArea in self.doorsArea.getMembers():
            s += '                  ' + str(doorArea) + '\n'
        s += '               外壁面積  : ' + '\n'
        for outsideWallArea in self.outsideWallsArea.getMembers():
            s += '                  ' + str(outsideWallArea) + '\n'
        s += '               外気床面積    : ' + str(self.outsideFloorArea) + '\n'
        s += '               界壁面積      : ' + str(self.insideWallArea) + '\n'
        s += '               界床面積      : ' + str(self.insideFloorArea) + '\n'     
        s += '               界天井面積    : ' + str(self.insideCeilingArea) + '\n'     
        s += '               床下床面積    : ' + str(self.crawlSpaceFloorArea) + '\n'
        s += '               熱橋長さ ψ 1 v w-w : ' + '\n'
        for l in self.l_psi_1v_ww:
            s += '                  ' + str(l) + '\n'
        s += '               熱橋長さ ψ 1 h w-r : ' + '\n'
        for l in self.l_psi_1h_wr:
            s += '                  ' + str(l) + '\n'
        s += '               熱橋長さ ψ 1 h w-f : ' + '\n'
        for l in self.l_psi_1h_wf:
            s += '                  ' + str(l) + '\n'
        s += '               熱橋長さ ψ 2 v w : ' + '\n'
        for l in self.l_psi_2v_w:
            s += '                  ' + str(l) + '\n'
        s += '               熱橋長さ ψ 2 h r : ' + '\n'
        for l in self.l_psi_2h_r:
            s += '                  ' + str(l) + '\n'
        s += '               熱橋長さ ψ 2 h w : ' + '\n'
        for l in self.l_psi_2h_w:
            s += '                  ' + str(l) + '\n'
        s += '               熱橋長さ ψ 2 h f : ' + '\n'
        for l in self.l_psi_2h_f:
            s += '                  ' + str(l) + '\n'
        s += '               熱橋長さ ψ 3 v ww : ' + '\n'
        for l in self.l_psi_3v_ww:
            s += '                  ' + str(l) + '\n'
        s += '               熱橋長さ ψ 3 h wr : ' + '\n'
        for l in self.l_psi_3h_wr:
            s += '                  ' + str(l) + '\n'
        s += '               熱橋長さ ψ 3 h wf : ' + '\n'
        for l in self.l_psi_3h_wf:
            s += '                  ' + str(l) + '\n'
        s += '               熱橋長さ ψ 1 h wuf : ' + '\n'
        for l in self.l_psi_1h_wuf:
            s += '                  ' + str(l) + '\n'
        s += '               熱橋長さ ψ 2 h uf : ' + '\n'
        for l in self.l_psi_2h_uf:
            s += '                  ' + str(l) + '\n'
    
        return s

In [74]:
d_General = {
    "Region"               : 6,    # 地域の区分
    "Structure"            : "RC", # 建物の構造
    "LevelsNumber"        : 3,    # 建物の階数
    "Height"               : 9.1,  # 建物の高さ(m)
    "OpeningMainDirection" : "S",  # 主開口部の方位
}

In [75]:
d_PrivateArea = {
    'HouseNumber'         : [4, 4, 4],                # 各階の住戸数
    'FloorArea'           : [162.64, 162.64, 162.64], # 各階の床面積の合計(m2)
    'PeriLengthOutside'   : [66.56, 66.56, 66.56],    # 各階の外周長 外気と接する部分(m)
    'PeriLengthInside'    : [0.0, 0.0, 0.0],          # 各階の外周長 空調された共用部や非住宅部と接する部分(m)
    'PeriLengthPrivate'   : [0.0, 0.0, 0.0],          # 各階の外周長 住戸と接する部分(住棟を複数の方位に分けて計算する場合のみ)(m)
    'WindowArea'          : [30.48, 30.64, 30.64],    # 各階の窓面積の合計(m2)
    'RoofArea'            : [0.0, 0.0, None],         # 各階(最上階を除く)の屋根面積の合計(m2)
    'OutsideFloorArea'    : [None, 0.0, 0.0],         # 各階(最下階を除く)の外気床の面積の合計(m2)
    'Wall'                : {                         # 外壁の仕様
        'UValue' : 0.785,    # 熱貫流率(U値)(W/m2K)
        'InsPos' : 'inside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
        'HBSpec' : 'spec1'   # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
    },
    'Roof'                : {                      # 屋根の仕様
        'UValue' : 0.417,     # 熱貫流率(U値)(W/m2K)
        'InsPos' : 'outside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
        'HBSpec' : 'spec1'    # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
    },
    'Floor'               : {                      # 最下階床の仕様
        'UValue' : 0.507,     # 熱貫流率(U値)(W/m2K)
        'InsPos' : 'outside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
        'HBSpec' : 'null'     # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
    },
    'InsideWallUValue'    : 2.34,                  # 界壁の熱貫流率(U値)(W/m2K)
    'InsideFloorUValue'   : 1.796,                 # 界床の熱貫流率(U値)(W/m2K)
    'WindowUValue'        : 4.65,                  # 窓の熱貫流率(U値)(W/m2K)
    'WindowEtaValue'      : 0.79                   # 窓の日射熱取得率(η値)    
}

In [76]:
d_PublicArea = {
    'FloorArea'          : [68.82, 59.7, 59.7], # 各階の共用部・非住宅部の床面積の合計(m2)
    'RoofArea'           : [9.125, 0.0, None],  # 各階(最上階を除く)の屋根面積の合計(m2)
    'OutsideFloorArea'   : [None, 0.0, 0.0],    # 各階(最下階を除く)の外気床の面積の合計(m2)
    'WallUValue'         : 0.00,                # 外壁の熱貫流率(U値)(W/m2K)
    'InsideFloorUValue'  : 0.00,                # 界床の熱貫流率(U値)(W/m2K)
    'OutsideFloorUValue' : 0.00,                # 外気床の熱貫流率(U値)(W/m2K)
    'WindowUValue'       : 0.00,                # 窓の熱貫流率(U値)(W/m2K)
    'WindowEtaValue'     : 0.00                 # 窓の日射熱取得率(η値)
}

In [77]:
d = {
    "General"     : d_General,
    "PrivateArea" : d_PrivateArea,
    "PublicArea"  : d_PublicArea
}

In [78]:
b = Building(d)
b.calcAL()

print( b )
p = b.privatearea
print( p )
for f in p.floors:
    print( f )
    for h in f.houses:
        print(h)

建物情報
      地域  : 6
      構造  : RC
      階数  : 3
      高さ  : 9.1
      主方位: S
   専有部
         壁  : U値:0.785, 断熱材位置:inside, 熱橋タイプ:spec1
         屋根: U値:0.417, 断熱材位置:outside, 熱橋タイプ:spec1
         床  : U値:0.507, 断熱材位置:outside, 熱橋タイプ:null
         界壁: U値:2.34
         界床: U値:1.796
         窓  : U値:4.65, η値:0.79
      階0
            住戸数    : 4
            床面積    : 162.64
            外周長    : 66.56
            窓面積    : 30.48
            屋根面積  : 0.0
            外気床面積: 0.0
            階高      : 3.033333333333333
            長辺長さ  : 27.328760451988813
            短辺長さ  : 5.9512395480111895
            窓面積    : 
               D0     : 18.336768
               D90    : 0.9144
               D180   : 10.314432
               D270   : 0.9144

         住戸:0
               屋根面積      : 0.0
               窓面積        : 
                  D0     : 4.584192
                  D90    : 0.9144
                  D180   : 2.578608
                  D270   : 0
               玄関ドア面積  : 
                  D0     